In [1]:
import dace

from dace import registry
from dace.sdfg.scope import ScopeSubgraphView
from dace.codegen.prettycode import CodeIOStream
from dace.codegen.targets.target import TargetCodeGenerator
from dace.codegen.targets.framecode import DaCeCodeGenerator
from dace.codegen.targets.cpp import sym2cpp
from IPython.display import Code
from dace.config import Config

In [2]:
@dace.program
def reduce_add_sync(mask: dace.uint32, value: dace.uint32):

    result = dace.define_local_scalar(dace.uint32)
    
    with dace.tasklet(dace.Language.CPP):
        inp_mask << mask
        inp_value << value
        out_result >> result
        """
        out_result = __reduce_add_sync(inp_mask, inp_value);
        """
    return result



@dace.program
def warpLevel(A: dace.uint32[1024] @ dace.dtypes.StorageType.GPU_Global, B: dace.uint32[1024] @ dace.dtypes.StorageType.GPU_Global):
    for _ in dace.map[0:1024:1024] @ dace.dtypes.ScheduleType.GPU_Device:
        for j in dace.map[0:1024] @ dace.dtypes.ScheduleType.GPU_ThreadBlock:

            for l, m, k in dace.map[0:4:2, 0:4, 0:2] @ dace.dtypes.ScheduleType.GPU_Warp:
                mask = 0xffffffff
                value = A[j]
                result = dace.define_local_scalar(dace.uint32)
                with dace.tasklet(dace.Language.CPP):
                    inp_mask << mask
                    inp_value << value
                    out_result >> result
                    """
                    out_result = __reduce_add_sync(inp_mask, inp_value);
                    """
                
                B[j] = result


In [3]:
sdfg = warpLevel.to_sdfg()

Code(sdfg.generate_code()[1].clean_code, language='cpp')

#include <cuda_runtime.h>
#include <dace/dace.h>


struct warpLevel_state_t {
    dace::cuda::Context *gpu_context;
};



DACE_EXPORTED int __dace_init_experimental_cuda(warpLevel_state_t *__state);
DACE_EXPORTED int __dace_exit_experimental_cuda(warpLevel_state_t *__state);

DACE_DFI void warpLevel_19_4_20_8_22_12_0_0_8(const dace::uint&  __tmp_24_24_r, dace::uint* __restrict__ __tmp_34_16_w) {
    dace::uint value;


    value = __tmp_24_24_r;
    {

        {
            dace::uint __out;

            ///////////////////
            // Tasklet code (assign_34_16)
            __out = __reduce_add_sync(4294967295U, value);
            ///////////////////

            __tmp_34_16_w[0] = __out;
        }

    }
}



int __dace_init_experimental_cuda(warpLevel_state_t *__state) {
    int count;

    // Check that we are able to run cuda code
    if (cudaGetDeviceCount(&count) != cudaSuccess)
    {
        printf("ERROR: GPU drivers are not configured or cuda-capable device "
               "not found\n");
        return 1;
    }
    if (count == 0)
    {
        printf("ERROR: No cuda-capable devices found\n");
        return 2;
    }

    // Initialize cuda before we run the application
    float *dev_X;
    DACE_GPU_CHECK(cudaMalloc((void **) &dev_X, 1));
    DACE_GPU_CHECK(cudaFree(dev_X));

    

    __state->gpu_context = new dace::cuda::Context(1, 1);

    // Create cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamCreateWithFlags(&__state->gpu_context->internal_streams[i], cudaStreamNonBlocking));
        __state->gpu_context->streams[i] = __state->gpu_context->internal_streams[i]; // Allow for externals to modify streams
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventCreateWithFlags(&__state->gpu_context->events[i], cudaEventDisableTiming));
    }

    

    return 0;
}

int __dace_exit_experimental_cuda(warpLevel_state_t *__state) {
    

    // Synchronize and check for CUDA errors
    int __err = static_cast<int>(__state->gpu_context->lasterror);
    if (__err == 0)
        __err = static_cast<int>(cudaDeviceSynchronize());

    // Destroy cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamDestroy(__state->gpu_context->internal_streams[i]));
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventDestroy(__state->gpu_context->events[i]));
    }

    delete __state->gpu_context;
    return __err;
}

DACE_EXPORTED bool __dace_gpu_set_stream(warpLevel_state_t *__state, int streamid, gpuStream_t stream)
{
    if (streamid < 0 || streamid >= 1)
        return false;

    __state->gpu_context->streams[streamid] = stream;

    return true;
}

DACE_EXPORTED void __dace_gpu_set_all_streams(warpLevel_state_t *__state, gpuStream_t stream)
{
    for (int i = 0; i < 1; ++i)
        __state->gpu_context->streams[i] = stream;
}

__global__ void warpLevel_19_0_0_0(dace::uint * __restrict__ A, dace::uint * __restrict__ B)
{ // Kernel scope (open 1)
    int _ = (1024 * blockIdx.x);
    { // ThreadBlock Scope (open 1)
        int j = threadIdx.x;
        { // WarpLevel Scope (open 1)
            int warpId_warpLevel_19_4_20_8_22_0_0_6 = (threadIdx.x) / 32;
            int k = warpId_warpLevel_19_4_20_8_22_0_0_6 % 2;
            int m = (warpId_warpLevel_19_4_20_8_22_0_0_6 / 2) % 4;
            int l = (warpId_warpLevel_19_4_20_8_22_0_0_6 / (2 * 4)) % 4;
            warpLevel_19_4_20_8_22_12_0_0_8(A[j], &B[j]);
        } // WarpLevel Scope (close 1)
    } // ThreadBlock Scope (close 1)
} // Kernel scope (close 1)


DACE_EXPORTED void __dace_runkernel_warpLevel_19_0_0_0(warpLevel_state_t *__state, dace::uint * __restrict__ A, dace::uint * __restrict__ B);
void __dace_runkernel_warpLevel_19_0_0_0(warpLevel_state_t *__state, dace::uint * __restrict__ A, dace::uint * __restrict__ B)
{


    void  *warpLevel_19_0_0_0_args[] = { (void *)&A, (void *)&B };
    gpuError_t __err = cudaLaunchKernel( (void*)warpLevel_19_0_0_0, dim3(1